In [ ]:
import os
import logging
from supabase import create_client
from  dotenv import load_dotenv
from postgrest import APIError
import time
import pandas as pd
load_dotenv()

# Test functions

In [ ]:
def validation(func): 
    """ Catch API error raised by supabase """ 
    def wrapper(*args, **kwargs): 
        try: 
            return func(*args, **kwargs) 
        except APIError as e: 
            logging.error(f"{e.message}") 
            return None 
        except Exception as e: 
            logging.error(f"Unexpected error: {e}") 
            return None 
    return wrapper 

In [ ]:
# Register with supabase 
# # IMPORTANT to use supabase built-in methods 
supabase = create_client( os.environ['PROJECT_URL'], os.environ['ANON_PUBLIC'] )

# Test case

Unauthorized

In [ ]:
@validation
def unauthorized_GET(table_name: str):
    response = supabase.table(table_name).select("*").execute()
    return {"row": f"{len(response.data)}"}
    
@validation
def unauthorized_POST(table_name: str, value: dict[str]):
    response = supabase.table(table_name).insert(value).execute()
    return {"row": f"{len(response.data)}"}

@validation
def unauthorized_DELETE(table_name: str, condition: dict[str, any]):
    query = supabase.table(table_name).delete()

    for key, val in condition.items():
        query = query.eq(key, val)

    response = query.execute()
    return {"row": len(response.data)}

@validation
def unauthorized_UPDATE(table_name: str, value: dict[str, any], condition: dict[str, any]):
    query = supabase.table(table_name).update(value)

    for key, val in condition.items():
        query = query.eq(key, val)

    response = query.execute()
    return {"row": len(response.data)}

Test unauthorized

In [ ]:
from datetime import datetime
print(f"run at: {datetime.now()}")

# fetch price table
unauthorized_GET("price")
# fetch stats table
unauthorized_GET("stats")
# insert to price
unauthorized_POST("price", value={'symbol': 'ABC'})
# delete on price
unauthorized_DELETE("price", condition={"symbol": "AAA"})
# adjust on price
unauthorized_UPDATE("price", value={'current_price': '2503'}, condition={'symbol': 'AAA'})

Authorized

In [ ]:
@validation
def authorized_GET(table_name: str, credentials: dict[str, str]):
    # Register to supabase's client object with authorized token
    _session = supabase.auth.sign_in_with_password(credentials)
    supabase.postgrest.auth(_session.session.access_token)
    
    response = supabase.table(table_name).select("*").execute()
    return {"row": f"{len(response.data)}"}

@validation
def authorized_POST(table_name: str, credentials: dict[str, str], value: dict[str]):
    _session = supabase.auth.sign_in_with_password(credentials)
    supabase.postgrest.auth(_session.session.access_token)
    
    response = supabase.table(table_name).insert(value).execute()
    print("Success")
    return {"row": f"{len(response.data)}"}

@validation
def authorized_DELETE(table_name: str, credentials: dict[str, str], condition: dict[str, any]):
    _session = supabase.auth.sign_in_with_password(credentials)
    supabase.postgrest.auth(_session.session.access_token)
    query = supabase.table(table_name).delete()

    for key, val in condition.items():
        query = query.eq(key, val)

    response = query.execute()
    print("Success")
    return {"row": len(response.data)}

@validation
def authorized_UPDATE(table_name: str, credentials: dict[str, str], value: dict[str, any], condition: dict[str, any]):
    _session = supabase.auth.sign_in_with_password(credentials)
    supabase.postgrest.auth(_session.session.access_token)
    query = supabase.table(table_name).update(value)

    for key, val in condition.items():
        query = query.eq(key, val)

    response = query.execute()
    print("Success")
    return {"row": len(response.data)}
    

Test authorized

In [ ]:
email = os.environ["email"]
password = os.environ["password"]
credentials = {"email": email, "password": password}
fake_credentials = {"email": "abc@gmail.com", "password": "1234567"}

In [ ]:
from datetime import datetime
print(f"run at: {datetime.now()}")
# fetch on stats
authorized_GET("stats", credentials=credentials)
# insert
authorized_POST("price", credentials=credentials, value={'symbol': 'ABC'})
# delete
authorized_DELETE("price", credentials=credentials, condition={"symbol": "AAA"})
# update
authorized_UPDATE("price", credentials=credentials, value={'current_price': '2503'}, condition={'symbol': 'AAA'})
# Invalid login
authorized_GET("price", credentials=fake_credentials)